# Briefe bereinigen

- notwendige Bibliotheken:

In [ ]:
import lxml        
from bs4 import BeautifulSoup as bs
import os
from pathlib import Path
import shutil

- der Pfad zu den Ordnern, in denen sich die zu bereinigenden Dateien befinden und in die die alten und die neuentstandenenen Dateien verschoben werden, wird definiert:

In [ ]:
pfad = r"ordner\\" # Ordner, in dem sich die zu bearbeitenden Dateien befinden
pfad2 = r"ordner2\\" # Ordner, in den die ursprünglichen Dateien verschoben werden
pfad3 = r"ordner3" # Ordner, in den die fertig bearbeiteten Dateien verschoben werden
os.chdir(pfad)

- Funktion, die einzelne XML-Dateien aufmacht und den XML-Inhalt parst: 

In [ ]:
def parse_file(f):
    with open(f, encoding="utf-8") as file:
        content = file.read()
        content = ''.join(content)
        bs_content = bs(content, 'lxml')
    return bs_content

- Funktion, die alte Dateien überschreibt:

In [ ]:
def make_file(element):
    with open(datei, 'w', encoding='utf-8') as fh_out:
        datei_neu = fh_out.write(str(element))
    return datei_neu

- Funktion, die in einem anderen Ordner eine neue TXT-Datei öffnet und den Inhalt des Briefes darin speichert:

In [ ]:
def make_txt_file(brief):
    ausgabe_datei = os.path.join(pfad3, f'{name}.txt')
    with open(ausgabe_datei, 'w', encoding='utf-8') as fh_out:
        datei = fh_out.write(brief)
    return datei

## Dateien umbenennen

- die Dateien werden eingelesen, der Inhalt geparst und die Dateien werden umbenannt
- der neue Name besteht aus dem Namen des Empfängers und einer generierten Nummer
- die alten XML-Dateien werden in einen anderen Ordner verschoben

In [ ]:
i=1
for datei in os.listdir(pfad): 
    f = os.path.join(pfad, datei) 
    if os.path.isfile(f): 

        bs_content = parse_file(f)
           
        for name in bs_content.find_all(attrs={'type': 'received'}):
            korresp = name.text
            korresp1 = korresp.strip()
            korresp2 = korresp1.split()
            with open (f'{korresp2[-1]}_{i}.xml', "w", encoding = "utf-8") as fh_out: 
                fh_out.write(str(bs_content))
                shutil.move(pfad + datei, pfad2)
                i+=1

## div herausfiltern

- in den umbenannten Dateien wird nach dem div-Element gesucht und die Dateien werden nur mit dem div-Element überschrieben:

In [ ]:
for datei in os.listdir(pfad): 
    f = os.path.join(pfad, datei) 
    if os.path.isfile(f): 
        bs_content = parse_file(f)
        for div in bs_content.find_all('div'): 
             datei_neu = make_file(div)

## note löschen

- es wird nach den Fußnoten gesucht und sie werden gelöscht:

In [ ]:
for datei in os.listdir(pfad): 
    f = os.path.join(pfad, datei) 
    if os.path.isfile(f): 
        bs_content = parse_file(f)
        for note in bs_content.find_all('note'):
            note.decompose()  
            datei_neu=make_file(bs_content)

## foreign löschen

- es wird nach Fremdwörtern gesucht und sie werden gelöscht:

In [ ]:
for datei in os.listdir(pfad): 
    f = os.path.join(pfad, datei) 
    if os.path.isfile(f): 
        bs_content = parse_file(f)
        for fremd in bs_content.find_all('foreign'):
            fremd.decompose()  
            datei_neu=make_file(bs_content)

## Sortieren nach einem Kriterium und umwandeln in TXT

- es wird nach dem opener-Element gesucht, eine Bedingung (in diesem Beispiel ist es das Wort "Freund", das im opener sein soll) definiert und, wenn die Bedingung erfüllt ist, wird der Briefinhalt in eine TXT-Datei in einem anderen Ordner (pfad3) gespeichert

In [ ]:
for datei in os.listdir(pfad):
    f = os.path.join(pfad, datei) 
    name, endung = datei.split('.')
    if os.path.isfile(f): 
        bs_content = parse_file(f)
        brief = bs_content.text
        for anrede in bs_content.find_all('opener'): 
            if 'Freund' in anrede.text and 'Herr' not in anrede.text and 'Kollege' not in anrede.text:
                datei = make_txt_file(brief)